# Minimal example using the database and DPsim

## Load credentials and sensitive data

In [ ]:
import json
# Opening JSON file
with open('../credentials/credentials.json', 'r') as openfile:
    # Reading from json file
    credentials = json.load(openfile)

## Configure the connection to the database and set the collections

In [ ]:
import pymongo
import sys
from pymongo import MongoClient
import pandas as pd

In [ ]:
db = MongoClient(credentials['pymongo_url'], 
                    credentials['pymongo_port'], 
                     username=credentials['pymongo_username'],
                     password=credentials['pymongo_password']).get_database(credentials['pymongo_database_name'])

In [ ]:
current_collection = eval(credentials['meter_current_name'])
power_collection = eval(credentials['meter_power_name'])
voltage_collection = eval(credentials['meter_voltage_name'])

## Import dependencies for Simulation

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import dpsimpy
from dpsim import matpower
import datetime

## Read the circuit file

In [ ]:
# List of assets
meters_assets_dict={'W0':[], 'W1':[], 'W2':[], 'W3': ['load2'], 'W4': ['load3'], 'W5':['load4'], 'W6': ['load4']}

In [ ]:
# Reader(mpc_file_path): mpc_file_path is relative to the Notebook dir
mpc_reader = matpower.Reader('../network_model/load_flow_pilot_district_mpc_struct_insp.mat', 'mpc')
system = mpc_reader.load_mpc()

In [ ]:
# Plot DPsim model
system

# Query to select dates to calculate the circuit results

In [ ]:
import ipywidgets as widgets
from ipywidgets import Layout, Box, Dropdown, Label


# available dates in collection:
power_collection_dates=sorted(power_collection.distinct("date"), key=lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
power_collection_meters=sorted(power_collection.distinct("id"))

start_date_selection= widgets.Select(
                                        options= power_collection_dates,
                                        value='2021-10-25',
                                        # rows=10,
                                        description='start date:',
                                        disabled=False
                                    )

display(start_date_selection)

end_date_selection= widgets.Select(
                                        options= power_collection_dates,
                                        value='2021-10-31',
                                        # rows=10,
                                        description='end date:',
                                        disabled=False
                                    )

display(end_date_selection)

## Get data for the selected timeframe

In [ ]:
user_requested_timestamps= pd.date_range(start_date_selection.value, end_date_selection.value, freq='5Min').tolist()
user_query = { "date": { "$gte": start_date_selection.value , "$lte": end_date_selection.value}}
user_query_power_df = pd.DataFrame(power_collection.find(user_query))
user_query_power_df

## Process the retrieved data

In [ ]:
db_columns=list(user_query_power_df.columns.values)
time_format = "%H:%M:%S"
db_times=[]
db_colums_withoutTime=[]
for col in db_columns:
    try:
        db_times.append(datetime.datetime.strptime(col, time_format).time())
        # print(f"{col} is the correct date string format.")
    except ValueError:
        db_colums_withoutTime.append(col)
        # print(f"{i} is the incorrect date string format.")

header=[times.strftime("%H:%M:%S") for times in db_times]
# header.insert(0,'date')

melted_df=pd.melt(user_query_power_df, id_vars=['date', 'id', 'power_type'], value_vars=header, var_name='time', value_name='power')
# user_query_power_df[user_query_power_df['power_type']=='positive_active'][header]
pivoted_df=pd.pivot_table(melted_df, values='power', columns='power_type', index=['date', 'time', 'id'])\
            .reset_index()
pivoted_df.columns.name=None
pivoted_df.insert(loc=0, column='datetime', value=pd.to_datetime(pivoted_df['date'] + ' ' + pivoted_df['time'], format="%Y-%m-%d %H:%M:%S"))
processed_power_df=pivoted_df.dropna()
processed_power_df

# Simulation service

## Set-up and start simulation with DPsim

In [ ]:
sim_name = 'pilot_uc6'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)
logger = dpsimpy.Logger(sim_name)

for node in system.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
    logger.log_attribute(node.name()+'.S', 's', node)

# Parametrize and run simulation
sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.info)
sim.set_system(system)
sim.set_time_step(1)
sim.set_final_time(len(user_requested_timestamps))
sim.set_domain(dpsimpy.Domain.SP)
sim.set_solver(dpsimpy.Solver.NRP)
sim.do_init_from_nodes_and_terminals(False)
sim.add_logger(logger)
sim.start()

In [ ]:
for component in system.components:
    print( component.name() )

## Helper functions to assign mesurements from the database to the simulator

In [ ]:
def get_meter_values(timestamp, meter):
    try:
        P= processed_power_df[(processed_power_df['id']==meter) & (processed_power_df['datetime']==timestamp)].iloc[0]['positive_active']
        Q= processed_power_df[(processed_power_df['id']==meter) & (processed_power_df['datetime']==timestamp)].iloc[0]['positive_reactive']
        return [P,Q]
    except:
        #print('[', timestamp, ']', '[', meter, ']', 'Warn: no available measurement dropping simulation step')
        raise ValueError("There is no available measurement in this timestep")
        #return ['nan', 'nan']

In [ ]:
kw_w= 1e3
def pq_assign_dpsim(timestamp):
    """
    take pq values of measurements and set the values in the DPSim to the corresponding asset
    """
    for k, v in meters_assets_dict.items():
        meter= k
        assets= v
        
        for asset in v:
            P_set=0
            Q_set =0
            [P_meter,Q_meter]=get_meter_values(timestamp, meter)

            P_set = P_set + (P_meter/3) # check if power is single or three phase, if RMS or absolute value etc.?
            Q_set = Q_set + (Q_meter/3)
        
            #print('[', timestamp, ']', '[', meter, ']', '[', asset, ']', 'Info: assigning [P,Q] values', P_set, Q_set)

            sim.get_idobj_attr(asset, 'P').set(P_set*kw_w)
            sim.get_idobj_attr(asset, 'Q').set(Q_set*kw_w)

## Main simulation loop

In [ ]:
sim_timesteps=[]
for dttm in user_requested_timestamps:
        try:
            pq_assign_dpsim(dttm)
            sim_timesteps.append(dttm.to_pydatetime())
            #print (dttm.to_pydatetime())
        except ValueError:
            pass
            #print('[',dttm,'] Current timestep was ignored due to one or more measurements missing')
        sim.next()
sim.stop()



# Results

## Load file with the results

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *

path = './logs/pilot_uc6/'
logName = 'pilot_uc6';
dataType = '.csv';
logFilename = path + logName + dataType;
ts_dpsim = read_timeseries_dpsim(logFilename)

## Enable the selection of the results for the nodes

In [ ]:
list_results_opt=list(ts_dpsim.keys())
result_selection= widgets.Select(
                                        options= list_results_opt,
                                        value= list_results_opt[7],
                                        # rows=10,
                                        description='result to show:',
                                        disabled=False
                                    )

display(result_selection)



## Plot the selected result

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#%matplotlib widget
X=sim_timesteps

result_selection.value
fig, axs = plt.subplots(2, 1, figsize=(10, 3), layout='constrained')

fig.suptitle("Time evolution of " + result_selection.value)


Y=ts_dpsim[result_selection.value].abs().values
axs[0].plot(X,Y)
axs[0].set_title('Amplitude')
axs[0].grid(True)
#axs[0].set_ylim(411,412)


Y=ts_dpsim[result_selection.value].phase().values
axs[1].plot(X,Y)
axs[1].grid(True)
axs[1].set_title('Angle')


axs[1].set_xlabel('Date')
axs[0].xaxis.set_major_locator(mdates.DayLocator())
axs[0].xaxis.set_minor_locator(mdates.HourLocator(byhour=(0,6,12,18,)))
axs[1].xaxis.set_major_locator(mdates.DayLocator())
axs[1].xaxis.set_minor_locator(mdates.HourLocator(byhour=(0,6,12,18,)))
plt.grid(True)


plt.show()

